In [5]:
import tensorflow as tf
import data_provider as data_provider
import config as config
import numpy as np
from tflearn.layers.conv import global_avg_pool
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import batch_norm, flatten
from tensorflow.contrib.framework import arg_scope
from tqdm import tqdm
from colorama import Fore
from PIL import Image
from tensorpack import *
import os

In [24]:
nb_block = 2
init_learning_rate = .1
dropout_rate = 0.2
nesterov_momentum = 0.9
weight_decay = 10

exp_decay_steps = 6000
exp_decay_rate = 1
regularization = .0001

#The Number of classes in dataset
class_num = 2

total_epochs = 2

os.environ["CUDA_VISIBLE_DEVICES"]="-1"  

In [41]:

def conv_layer(input, filter, kernel, stride=1, layer_name="conv"):
    with tf.name_scope(layer_name):
        #regularizer controlled here
        
        network = tf.layers.conv2d(inputs=input, filters=filter, kernel_size=kernel, strides=stride, padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=regularization))
        return network

def Global_Average_Pooling(x, stride=1):
    

    return global_avg_pool(x, name='Global_avg_pooling')


def Batch_Normalization(x, training, scope):
    with arg_scope([batch_norm],
                   scope=scope,
                   updates_collections=None,
                   decay=0.9,
                   center=True,
                   scale=True,
                   zero_debias_moving_mean=True) :
        return tf.cond(training,
                       lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                       lambda : batch_norm(inputs=x, is_training=training, reuse=True))

def Drop_out(x, rate, training) :
    return tf.layers.dropout(inputs=x, rate=rate, training=training)

def Relu(x):
    return tf.nn.relu(x)

def Average_pooling(x, pool_size=[2,2], stride=2, padding='SAME'):
    return tf.layers.average_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)


def Max_Pooling(x, pool_sizes, strides, padding='SAME'):
    return tf.layers.max_pooling2d(inputs=x, pool_size=pool_sizes, strides=strides, padding=padding)

def Concatenation(layers) :
    return tf.concat(layers, axis=3)

def Linear(x,name) :
    return tf.layers.dense(inputs=x, units=class_num, name=name)



class DenseNet():
    def __init__(self, x, filters, training):
        self.filters = filters
        self.training = training
        self.model = self.Dense_net(x)


    def bottleneck_layer(self, i, scope):
        # print(x)
        with tf.name_scope(scope):


            x = Batch_Normalization(i, training=self.training, scope=scope+'_batch2')
            x = Relu(x)
            x = conv_layer(x, filter=self.filters, kernel=[3,3], layer_name=scope+'_conv2')
            x = Drop_out(x, rate=dropout_rate, training=self.training)
            i = tf.concat([x,i],3)

            # print(x)

            return i

    def transition_layer(self, x, scope):
        shape = x.get_shape().as_list()
        in_channel = shape[3]
        with tf.name_scope(scope):
            x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
            x = Relu(x)
            x = conv_layer(x, in_channel, kernel=[1,1], layer_name=scope+'_conv1')
            x = Drop_out(x, rate=dropout_rate, training=self.training)
            x = Average_pooling(x, pool_size=[2,2], stride=2)

            return x
    
    def Dense_net(self, input_x):
        
        x = conv_layer(input_x, 16, kernel=3)
        x = Max_Pooling(x, pool_sizes=2, strides = 2)



        #for i in range(self.nb_blocks) :
        # 6 -> 12 -> 18
        with tf.name_scope('block'+str(1)):
            for i in range(5):
            
                x = self.bottleneck_layer(x, scope='block1' + '_bottleN_' + str(i + 1))
                
            x = self.transition_layer(x, scope='trans_'+str(1))
        with tf.name_scope('block'+str(2)):
            for i in range(5):
                x = self.bottleneck_layer(x, scope='block2' + '_bottleN_' + str(i + 1))

            x = self.transition_layer(x, scope='trans_'+str(2))

        with tf.name_scope('block'+str(3)):
            
            
            for i in range(5):
                
                x = self.bottleneck_layer(x, scope='block3' + '_bottleN_' + str(i + 1))

            

        #x = self.dense_block(input_x=x, nb_layers=3, layer_name='block'+str(3))
        #x = self.transition_layer(x, scope='trans_'+str(i))



        """
        x = self.dense_block(input_x=x, nb_layers=6, layer_name='dense_1')
        x = self.transition_layer(x, scope='trans_1')

        x = self.dense_block(input_x=x, nb_layers=12, layer_name='dense_2')
        x = self.transition_layer(x, scope='trans_2')

        x = self.dense_block(input_x=x, nb_layers=48, layer_name='dense_3')
        x = self.transition_layer(x, scope='trans_3')
        """

        #x = self.dense_block(input_x=x, nb_layers=4, layer_name='dense_final')

        # 20 Layer
        x = Batch_Normalization(x, training=self.training, scope='linear_batch')
        x = Relu(x)
        x = Global_Average_Pooling(x)
        x = flatten(x)
        x = Linear(x,'linear0')
        return x
    
def densenet_train():
    image_batch_placeholder = tf.placeholder("float32", shape=[None, 256, 256, 3])
    label_batch_placeholder = tf.placeholder(tf.float32, shape=[None, 2])
    #   if_training_placeholder = tf.placeholder(tf.bool, shape=[])

    image_batch, label_batch = data_provider.feed_data(if_random = True, if_training = True)
    



    training_flag = tf.placeholder(tf.bool)

    logits = DenseNet(x = image_batch_placeholder, filters = 12, training=training_flag).model

    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=label_batch_placeholder, logits=logits))
    #wd_decay = tf.multiply(1e-4, regularize_cost('.*/W', tf.nn.l2_loss), name='wd_decay')
    #add_moving_summary(loss,wd_decay)
    logits_batch = tf.to_int64(tf.arg_max(logits, dimension = 1))

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label_batch_placeholder, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
    #loss = tf.losses.mean_squared_error(labels=label_batch_placeholder, predictions=logits)

    tf.summary.scalar('loss', loss) # create a summary for training loss

    regularzation_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #tf.summary.scalar('regularzation_loss', regularzation_loss)
    reg_constant = 0.01
    
    total_loss = sum(regularzation_loss) + loss
    tf.summary.scalar('total_loss', total_loss)

    global_step = tf.Variable(0, name='global_step', trainable=False)

    #can change to constant here. 
    learning_rate = tf.train.exponential_decay(learning_rate=init_learning_rate,
                                               global_step=global_step,
                                               decay_steps= exp_decay_steps,
                                               decay_rate= exp_decay_rate,
                                               staircase=True)
    tf.summary.scalar('learning_rate', learning_rate)

    #can change to ADAM here or gradient descent regular 
    
    if(momentum_type == "Nesterov"):
        train_step = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum=nesterov_momentum,use_nesterov=True).minimize(total_loss, global_step=global_step)
    if(momentum_type == "ADAM"):
        train_step = tf.train.AdamOptimizer(learning_rate=init_learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8,
               use_locking=False, name="Adam").minimize(total_loss, global_step=global_step)
    if(momentum_type == "ADAGRAD"):
        train_step = tf.train.AdagradOptimizer(init_learning_rate, initial_accumulator_value=0.1,
               use_locking=False, name="Adagrad").minimize(total_loss, global_step=global_step)

    summary_op = tf.summary.merge_all()  # merge all summaries into a single "operation" which we can execute in a session

    saver = tf.train.Saver(max_to_keep=100)
    all_trainable_vars = tf.reduce_sum([tf.reduce_prod(v.shape) for v in tf.trainable_variables()])

    #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.666)


    config = tf.ConfigProto(log_device_placement=False)
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)

    summary_writer = tf.summary.FileWriter("./log", sess.graph)

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    accuracy_accu = 0

    checkpoint = None
    #if restoring from checkpoint uncomment this
    #checkpoint = tf.train.get_checkpoint_state("./brehis_ici_model_binary")
    if(checkpoint != None):
        tf.logging.info("Restoring full model from checkpoint file %s",checkpoint.model_checkpoint_path)
        saver.restore(sess, checkpoint.model_checkpoint_path)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess = sess)

    check_points = 20
    for epoch in range(0,total_epochs):
        print("epoch is" + str(epoch))
        for check_point in tqdm(range(check_points),    
        bar_format="{l_bar}%s{bar}%s{r_bar}" % (Fore.BLUE, Fore.RESET)):

            image_batch_train, label_batch_train = sess.run([image_batch, label_batch])

            _, training_loss, _global_step, summary = sess.run([train_step, loss, global_step, summary_op],
                                                 feed_dict={image_batch_placeholder: image_batch_train,
                                                            label_batch_placeholder: label_batch_train,
                                                            training_flag: True})
            #bool(check_point != 0)
            if(bool(check_point%10 == 0)):
                print(_)
                print("training checkpoint: ", check_point + epoch * check_points)
                print("training loss: ", training_loss)
                print("Total weight:", sess.run(all_trainable_vars))
                print("Current Learning-Rate:", sess.run(learning_rate))

                summary_writer.add_summary(summary, _global_step)

        
        #if you want to save the checkpoint uncomment this
        #saver.save(sess, "./brehis_ici_model_binary/ch.ckpt", _global_step)
        if _global_step >= 240000:
            break
        
        
        


    coord.request_stop()
    coord.join(threads)
    sess.close()
    return 0

In [18]:
def main():
    tf.reset_default_graph()
    densenet_train()

In [14]:
momentum_type = "ADAM"
main()

Filling queue with 9600 images before starting to train. This will take some time.
INFO:tensorflow:Restoring full model from checkpoint file ./brehis_ici_model_binary\ch.ckpt-80
INFO:tensorflow:Restoring parameters from ./brehis_ici_model_binary\ch.ckpt-80
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  13.208616
Total weight: 191322
Current Learning-Rate: 0.1


 50%|█████     | 10/20 [09:57<09:43, 58.37s/it]

None
training checkpoint:  10
training loss:  26.060658
Total weight: 191322
Current Learning-Rate: 0.1


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  22.157194
Total weight: 191322
Current Learning-Rate: 0.1


 50%|█████     | 10/20 [09:33<09:30, 57.06s/it]

None
training checkpoint:  30
training loss:  23.216454
Total weight: 191322
Current Learning-Rate: 0.1


100%|██████████| 20/20 [19:02<00:00, 56.51s/it]


In [15]:
init_learning_rate = 1.0
momentum_type = "ADAM"
main()


Filling queue with 9600 images before starting to train. This will take some time.
INFO:tensorflow:Restoring full model from checkpoint file ./brehis_ici_model_binary\ch.ckpt-120
INFO:tensorflow:Restoring parameters from ./brehis_ici_model_binary\ch.ckpt-120
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  20.708124
Total weight: 191322
Current Learning-Rate: 1.0


 50%|█████     | 10/20 [09:53<09:47, 58.76s/it]

None
training checkpoint:  10
training loss:  48.04075
Total weight: 191322
Current Learning-Rate: 1.0


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  134.737
Total weight: 191322
Current Learning-Rate: 1.0


 50%|█████     | 10/20 [09:39<09:35, 57.58s/it]

None
training checkpoint:  30
training loss:  76.29205
Total weight: 191322
Current Learning-Rate: 1.0


100%|██████████| 20/20 [19:17<00:00, 57.86s/it]


In [45]:
init_learning_rate = .001
momentum_type = "Nesterov"
main()


Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  38.1938
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [14:14<12:19, 73.98s/it]

None
training checkpoint:  10
training loss:  15.957069
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  21.710157
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [11:05<11:04, 66.47s/it]

None
training checkpoint:  30
training loss:  26.298061
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [22:17<00:00, 65.10s/it]


In [25]:
init_learning_rate = .001
regularization = .001
momentum_type = "ADAM"
main()


Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  50.262753
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:59<09:48, 58.82s/it]

None
training checkpoint:  10
training loss:  24.891222
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  19.50465
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:41<09:41, 58.19s/it]

None
training checkpoint:  30
training loss:  20.665682
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:30<00:00, 58.25s/it]


In [26]:
init_learning_rate = .001
regularization = .001
momentum_type = "Nesterov"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  33.01615
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:59<09:49, 58.94s/it]

None
training checkpoint:  10
training loss:  22.585104
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  10.085077
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:43<09:41, 58.17s/it]

None
training checkpoint:  30
training loss:  27.59055
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:29<00:00, 58.91s/it]


In [28]:
init_learning_rate = .001
regularization = .001
momentum_type = "ADAGRAD"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  38.40235
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:18<10:01, 60.16s/it]

None
training checkpoint:  10
training loss:  24.261717
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  24.520802
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:46<09:47, 58.72s/it]

None
training checkpoint:  30
training loss:  22.042383
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:33<00:00, 58.57s/it]


In [34]:
init_learning_rate = .001
regularization = .01
momentum_type = "ADAM"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  58.831734
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:09<10:05, 60.55s/it]

None
training checkpoint:  10
training loss:  19.950806
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  23.764097
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:00<09:58, 59.84s/it]

None
training checkpoint:  30
training loss:  17.118456
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:56<00:00, 59.17s/it]


In [33]:
init_learning_rate = .001
regularization = .01
momentum_type = "Nesterov"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  33.285606
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:04<09:58, 59.82s/it]

None
training checkpoint:  10
training loss:  30.744848
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  24.635479
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:06<10:07, 60.79s/it]

None
training checkpoint:  30
training loss:  16.470516
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [20:04<00:00, 60.37s/it]


In [32]:
init_learning_rate = .001
regularization = .01
momentum_type = "ADAGRAD"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  32.166283
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:13<10:10, 61.05s/it]

None
training checkpoint:  10
training loss:  21.327787
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  18.086004
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:57<09:59, 59.92s/it]

None
training checkpoint:  30
training loss:  15.82132
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:49<00:00, 58.50s/it]


In [35]:
init_learning_rate = .001
regularization = .1
momentum_type = "ADAM"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  45.232258
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:11<10:02, 60.24s/it]

None
training checkpoint:  10
training loss:  13.656053
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  22.187944
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:46<09:47, 58.78s/it]

None
training checkpoint:  30
training loss:  15.338309
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:43<00:00, 60.09s/it]


In [36]:
init_learning_rate = .001
regularization = .1
momentum_type = "Nesterov"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  37.52027
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:52<09:46, 58.68s/it]

None
training checkpoint:  10
training loss:  20.298044
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  24.766613
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:57<09:56, 59.61s/it]

None
training checkpoint:  30
training loss:  29.773773
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:25<00:00, 57.88s/it]


In [38]:
init_learning_rate = .001
regularization = .1
momentum_type = "ADAGRAD"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  35.024323
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:09<09:54, 59.43s/it]

None
training checkpoint:  10
training loss:  21.662045
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  25.997416
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [09:47<09:40, 58.07s/it]

None
training checkpoint:  30
training loss:  18.493244
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:42<00:00, 59.28s/it]


In [42]:
init_learning_rate = .001
regularization = .01
momentum_type = "ADAGRAD"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  43.014656
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:21<10:14, 61.49s/it]

None
training checkpoint:  10
training loss:  25.425123
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  19.266441
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:00<10:01, 60.18s/it]

None
training checkpoint:  30
training loss:  18.027332
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [20:02<00:00, 60.82s/it]


In [43]:
init_learning_rate = .001
regularization = .01
momentum_type = "Nesterov"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  49.444435
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:11<10:02, 60.23s/it]

None
training checkpoint:  10
training loss:  24.637245
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  21.94733
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:01<10:01, 60.15s/it]

None
training checkpoint:  30
training loss:  21.196907
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [19:54<00:00, 59.49s/it]


In [44]:
init_learning_rate = .001
regularization = .1
momentum_type = "ADAM"
main()

Filling queue with 9600 images before starting to train. This will take some time.
epoch is0


  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  0
training loss:  30.302864
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:23<10:09, 60.98s/it]

None
training checkpoint:  10
training loss:  24.315826
Total weight: 191322
Current Learning-Rate: 0.001


...

  0%|          | 0/20 [00:00<?, ?it/s]

None
training checkpoint:  20
training loss:  17.273472
Total weight: 191322
Current Learning-Rate: 0.001


 50%|█████     | 10/20 [10:15<10:41, 64.13s/it]

None
training checkpoint:  30
training loss:  21.928492
Total weight: 191322
Current Learning-Rate: 0.001


100%|██████████| 20/20 [21:08<00:00, 64.99s/it]
